In [1]:
import pandas as pd
import numpy as np
from arch import arch_model
from statsmodels import regression
from scipy import linalg
import re

In [3]:
data = pd.read_csv("oil_futures.csv")
log_r = data.log_return
log_r = log_r[log_r!=0]

In [28]:
test_oil = log_r[970:]
training_oil = log_r[:-30]
garch_testoil = arch_model(test_oil*1000, p=1,q=1)
res_testoil = garch_testoil.fit(update_freq=5)

Iteration:      5,   Func. Count:     31,   Neg. LLF: 215.37648921264974
Iteration:     10,   Func. Count:     61,   Neg. LLF: 215.31671423599306
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 215.31500769293996
            Iterations: 12
            Function evaluations: 73
            Gradient evaluations: 12


In [33]:
def report_garch(test,train,p,q,scalar):
    #Training
    garch_train = arch_model(train*scalar, p=p,q=q)
    res_train = garch_train.fit(update_freq=5)
    mu, omega, alpha, beta = res_train.params
    
    #Test
    garch_test = arch_model(test*scalar,p=p,q=q)
    res_test = garch_test.fit(update_freq=5)
    
    #Prediction
    pred = np.sqrt(1/scalar*(omega+alpha*res_test.resid**2+res_test.conditional_volatility**2*beta))
    
    # MZ OLS Regression
    complete = np.c_[np.ones(len(pred)), pred]
    MZ_test = regression.linear_model.OLS(test**2, complete)
    result = MZ_test.fit()
    
    return (result.params)

# MZ Test for Oil Data

In [34]:
report_garch(test_oil,training_oil, p=1,q=1,scalar=1000)

Iteration:      5,   Func. Count:     35,   Neg. LLF: 3541.3886885016436
Iteration:     10,   Func. Count:     70,   Neg. LLF: 3541.156418276011
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3541.1498898988584
            Iterations: 12
            Function evaluations: 83
            Gradient evaluations: 12
Iteration:      5,   Func. Count:     31,   Neg. LLF: 215.37648921264974
Iteration:     10,   Func. Count:     61,   Neg. LLF: 215.31671423599306
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 215.31500769293996
            Iterations: 12
            Function evaluations: 73
            Gradient evaluations: 12


const   -2.240449
x1       0.195755
dtype: float64

# MZ Test for Exchange Rate Data

In [65]:
dat = pd.read_csv("DEXJPUS.csv")
dat = dat.loc[dat.DEXJPUS != "."].DEXJPUS.astype(float)
dat = np.array(dat)
dat = np.log(dat[1:])-np.log(dat[0:-1])
dat = dat[:,np.newaxis]

In [52]:
train = pd.DataFrame(dat[:-30])
test = pd.DataFrame(dat[-30:])

In [42]:
test.shape

(30, 1)

In [57]:
report_garch(train,test,p=1,q=1,scalar=100)

Iteration:      5,   Func. Count:     34,   Neg. LLF: 31.331296351240184
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 31.32840424779346
            Iterations: 8
            Function evaluations: 53
            Gradient evaluations: 8
Iteration:      5,   Func. Count:     42,   Neg. LLF: 6562.970753413042
Iteration:     10,   Func. Count:     74,   Neg. LLF: 6558.307956580733
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6558.307959331568
            Iterations: 11
            Function evaluations: 80
            Gradient evaluations: 11


const   -0.000044
x1       0.001342
dtype: float64

# Posterior Check for Oil Data

In [17]:
garch_complete = arch_model(log_r*1000,p=1,q=1)
res_complete = garch_complete.fit(update_freq=5)
fore_result = res_complete.forecast()
(fore_result.mean/1000)[-1:]

Iteration:      5,   Func. Count:     31,   Neg. LLF: 5579.6220064688105
Iteration:     10,   Func. Count:     61,   Neg. LLF: 5578.24538662792
Iteration:     15,   Func. Count:    107,   Neg. LLF: 5248.694186650964
Iteration:     20,   Func. Count:    141,   Neg. LLF: 5136.633117818144
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 5133.7662165158945
            Iterations: 24
            Function evaluations: 164
            Gradient evaluations: 23


,h.1
1000,-0.000847


# Posterior Check for Exchange Rate

In [59]:
train.shape

(7268, 1)

In [72]:
garch_exchange = arch_model(dat*100,p=1,q=1)
res_exchange = garch_exchange.fit(update_freq=5)
forecast_exchange = res_exchange.forecast()
(forecast_exchange.mean)[-1:]

Iteration:      5,   Func. Count:     42,   Neg. LLF: 6599.211185452592
Iteration:     10,   Func. Count:     74,   Neg. LLF: 6594.862137239937
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6594.862141348218
            Iterations: 10
            Function evaluations: 74
            Gradient evaluations: 10


,h.1
7297,-0.002525


# One-Day Forward Volatility

In [75]:
forecasted_vol = 0.001 * np.sqrt(res.params['omega'] + res.params['alpha[1]'] * res.resid**2 + res.conditional_volatility**2 * res.params['beta[1]'])

In [76]:
model_fore = res.conditional_volatility*0.001

In [77]:
complete = np.c_[np.ones(len(forecasted_vol)),model_fore]

In [78]:
MZ_test = regression.linear_model.OLS(log_r**2,complete)

In [79]:
result = MZ_test.fit()

In [80]:
result.params

const   -0.012969
x1       0.380222
dtype: float64

In [81]:
dat = pd.read_csv("DEXJPUS.csv")

In [82]:
for i,v in enumerate(dat.DATE):
    if re.sub("-.*","",v) == "1990":
        break
    
dat = dat.loc[dat.DEXJPUS != "."].DEXJPUS.astype(float)
dat = np.array(dat)
dat = np.log(dat[1:])-np.log(dat[0:-1])
dat = dat[:,np.newaxis]
train = dat[:i]
test = dat[i:]

In [83]:
train.shape

(4326, 1)

In [84]:
garch11_train = arch_model(train*100, p=1,q=1)
res_train = garch11_train.fit(update_freq=5)

Iteration:      5,   Func. Count:     41,   Neg. LLF: 3493.691460700021
Iteration:     10,   Func. Count:     72,   Neg. LLF: 3479.226118808358
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3479.226126061366
            Iterations: 10
            Function evaluations: 72
            Gradient evaluations: 10


In [85]:
# Parameters of the training set
mu, omega, alpha, beta = res_train.params

In [86]:
#To extract residuals and the conditonal volatilities of the test set
garch11_test = arch_model(test*100, p=1,q=1)
res_test = garch11_test.fit(update_freq=5)

Iteration:      5,   Func. Count:     40,   Neg. LLF: 3074.762431321689
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3074.719556857562
            Iterations: 9
            Function evaluations: 67
            Gradient evaluations: 9


In [107]:
#shifted = pd.DataFrame(res_test.conditional_volatility)
#shifted = shifted.shift(periods=1)
#2971 obs
#shifted = shifted[1:]

In [131]:
shifted_residual = res_test.resid[:-1].reshape(-1,1)

In [146]:
# Calculating the one-step forward prediction on the volatility.
#pred = 0.01*np.sqrt(omega+alpha*shifted_residual**2+shifted**2*beta)

In [143]:
# Calculating the one-step forward prediction on the volatility.
pred = 0.1*np.sqrt(omega+alpha*res_test.resid**2+res_test.conditional_volatility**2*beta)
pred = pred.reshape(-1,1)

In [147]:
complete2 = np.c_[np.ones(len(pred)),pred]

In [148]:
MZ_test_DEXJPUS = regression.linear_model.OLS(test**2,complete2)

In [149]:
result2 = MZ_test_DEXJPUS.fit()

In [150]:
result2.params

array([-6.97565045e-05,  1.74525895e-02])